In [ ]:
from troposphere import Template, Ref, FindInMap, Parameter, GetAtt, Tags, Output
import troposphere.ec2 as ec2
resources = set()

In [ ]:
t = Template()

# Mappings

In [ ]:
t.add_mapping('RegionMap', { 'us-east-1': {'AMI': 'ami-b73b63a0'}
                           , 'us-east-2': {'AMI': 'ami-58277d3d'}
                           , 'us-west-1': {'AMI': 'ami-23e8a343'}
                           , 'us-west-2': {'AMI': 'ami-5ec1673e'}
                           })

# Params

In [ ]:
keyname_param = Parameter('KeyName',
                          Description='Name of existing keypair',
                          Type='String')
t.add_parameter(keyname_param);

# Resources

In [ ]:
vpc = ec2.VPC('myVPC', CidrBlock='10.0.0.0/16')
resources.add(vpc)

In [ ]:
igw = ec2.InternetGateway('myIGW')
resources.add(igw)

In [ ]:
igw_attachment = ec2.VPCGatewayAttachment('igwAttachment', InternetGatewayId=Ref(igw), VpcId=Ref(vpc))
resources.add(igw_attachment)

In [ ]:
vpc_route_table = ec2.RouteTable('vpcRouteTable', VpcId=Ref(vpc))
resources.add(vpc_route_table)

In [ ]:
igw_route = ec2.Route('igwRoute',
                      DestinationCidrBlock='0.0.0.0/0',
                      GatewayId=Ref(igw),
                      RouteTableId=Ref(vpc_route_table),
                      DependsOn='myIGW')
resources.add(igw_route)

In [ ]:
webDMZ = ec2.Subnet('webDMZ', VpcId=Ref(vpc), CidrBlock='10.0.1.0/24', MapPublicIpOnLaunch=True)
resources.add(webDMZ)

In [ ]:
route_table_association = ec2.SubnetRouteTableAssociation('AssociateTable',
                                                          RouteTableId=Ref(vpc_route_table),
                                                          SubnetId=Ref(webDMZ))
resources.add(route_table_association)

In [ ]:
rules = list()
ports_to_open = [22, 80, 443]
for port in ports_to_open:
    rules.append(ec2.SecurityGroupRule(CidrIp='0.0.0.0/0', FromPort=port, ToPort=port, IpProtocol='tcp'))

web_sg = ec2.SecurityGroup('webSG', 
                           VpcId=Ref(vpc), 
                           GroupDescription='sg to allow web traffic/ssh',
                           SecurityGroupIngress=rules)
resources.add(web_sg)

In [ ]:
web1 = ec2.Instance('web1',
                    ImageId=FindInMap('RegionMap', Ref('AWS::Region'), 'AMI'),
                    InstanceType='t2.micro',
                    SubnetId=Ref(webDMZ),
                    SecurityGroupIds=[GetAtt(web_sg, 'GroupId')],
                    KeyName=Ref(keyname_param),
                    Tags=Tags(Name='web1'))
resources.add(web1)

# Outputs

In [ ]:
t.add_output(Output('PublicIP',
                   Description='Web node public IP',
                   Value=GetAtt(web1, 'PublicIp')));

# Build

In [ ]:
for r in resources:
    t.add_resource(r)

with open('trop.json', 'w') as fd:
    fd.write(t.to_json())